In [61]:
import numpy as np
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import molmass

In [95]:
# reading_data
data_wb = xlrd.open_workbook('CHE223-Assignment1 for 2023-data sets up to 2019.xls')

data = pd.DataFrame({'Year': [], 'Ion': [], 'Conc (mg/L)': [], 'Relative Standard Deviation (RSD)': [], 'Postal Code': []})

# Reading Ions

ion_dict = {}
postal_code = {}

for sheet in data_wb.sheets():

    year = ''
    for char in sheet.name:
        if char.isdigit(): year += char
    year = int(year)

    ions = []
    for row in np.arange(11):
        for col in np.arange(sheet.ncols):
            value = sheet.cell_value(row,col)

            if type(value) is str:
                if len(value)<=3:
                    try :
                        molmass.Formula(value)._elements
                        ions.append((year, col, row, value))
                    except: pass
                elif 'postal' in value.lower():
                    postal_code[sheet] = col

    ion_dict[sheet] = ions
    

In [113]:
# iterate through rows and record values based on (column, ion) pairs
notnum = 0
for sheet in data_wb.sheets():
    # print(sheet)
    for i in np.arange(len(ion_dict[sheet])):
        for row in np.arange(ion_dict[sheet][i][2]+1, sheet.nrows):

            value = sheet.cell_value(row,ion_dict[sheet][i][1])



            ### data_cleanup


            if type(value) is str and value.strip() != '':
                notnum += 1
                value = value.strip('ppm')
                print(value)
            
# row => [ion_dict[sheet][0] {year}, ion_dict[sheet][3] {ion}, value, pocode]
notnum

1.288
3.83x10^4
3.05*10^4
35.42
1.25x10^6
1.16*10^6
0.704
1.29x10^4
19.735
7.40x10^5
Missing
Missing
Missing
Missing
Missing
Missing
Missing
issing
Missing
Missing
issing
N/A
Missing
issing
Missing
Missing
Missing 
Missing 
Missing
Missing
issing
Missing
issing
Missing
N/A
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
issing
issing
Missing
issing
Missing
Missing
Missing
Missing
Missing
Missing
issing
Missing
issing
Missing
10,73
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
Ca
ICP results
-
-
-
-
Cu
-
-
-
-
Fe
ICP results
-
K
Mg
ICP results
-
-
-
-
Mn
Na
ICP results
-
-
-
-
Pb
-
-
-
-
Zn
ICP results
-
-
-
Si
-
-
-
-
Al
ICP results
N/A
0.562
N/A
N/A
0. 5579
F
59.7
Cl
IC results
0.7107
NO3
26.87
SO4
Ca
ICP results
-
NA
undetected
N/A
Cu
-
NA
undetected
N/A
Fe
NA
undetected
K
Mg
ICP results
-
NA
undetected
N/A
Mn
Na
-
NA
undetected
N/A
Pb
-
NA
undetected
N/A
Zn
ICP results
-
NA
undetected
N/A
Si
-
NA
undetected
N/A
Al
0..357
-
Missing
MISSING
Not Found
0..45
0.45
1.45min
M

255

In [ ]:
data = pd.DataFrame({'Year': [], 'Ion': [], 'Conc (mg/L)': [], 'Postal Code': []})


Notes:
- for missing data use <code>np.nan</code>